In [4]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import ast

Sistema de recomendación item-item:

    def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

Ruta relativa para traer el dataset limpio que vamos a utilizar y creamos un dataframe para trabajar

In [2]:
# Creamos el dataframe
games = pd.read_csv('steam_games_cleaned.csv')
# Solo nos quedamos con las columnas que vamos a necesitar
modelo_df = games.loc[:, ["specs", "item_id",'name']]

In [3]:
# Asignamos la columna 'specs' a una variable para luego tratarla y limpiar caracteres
specs_variable = modelo_df['specs'].copy()
specs_variable

0                                        ['Single-player']
1        ['Single-player', 'Multi-player', 'Online Mult...
2        ['Single-player', 'Multi-player', 'Online Mult...
3                                        ['Single-player']
4        ['Single-player', 'Full controller support', '...
                               ...                        
32128              ['Single-player', 'Steam Achievements']
32129    ['Single-player', 'Steam Achievements', 'Steam...
32130    ['Single-player', 'Steam Achievements', 'Steam...
32131    ['Single-player', 'Steam Achievements', 'Steam...
32132    ['Single-player', 'Stats', 'Steam Leaderboards...
Name: specs, Length: 32133, dtype: object

In [4]:
# Convertimos a cadena antes de reemplazar los corchetes que queremos sacar
specs_variable = specs_variable.apply(lambda x: str(x).replace('[', '').replace(']', ''))
specs_variable

0                                          'Single-player'
1        'Single-player', 'Multi-player', 'Online Multi...
2        'Single-player', 'Multi-player', 'Online Multi...
3                                          'Single-player'
4        'Single-player', 'Full controller support', 'H...
                               ...                        
32128                'Single-player', 'Steam Achievements'
32129    'Single-player', 'Steam Achievements', 'Steam ...
32130    'Single-player', 'Steam Achievements', 'Steam ...
32131    'Single-player', 'Steam Achievements', 'Steam ...
32132    'Single-player', 'Stats', 'Steam Leaderboards'...
Name: specs, Length: 32133, dtype: object

In [5]:
# Suponiendo que specs_variable es tu serie con las cadenas
specs_variable = specs_variable.str.replace("'", '')
specs_variable

0                                            Single-player
1        Single-player, Multi-player, Online Multi-Play...
2        Single-player, Multi-player, Online Multi-Play...
3                                            Single-player
4        Single-player, Full controller support, HTC Vi...
                               ...                        
32128                    Single-player, Steam Achievements
32129    Single-player, Steam Achievements, Steam Cloud...
32130    Single-player, Steam Achievements, Steam Tradi...
32131       Single-player, Steam Achievements, Steam Cloud
32132    Single-player, Stats, Steam Leaderboards, HTC ...
Name: specs, Length: 32133, dtype: object

In [6]:



# Reemplazamos la columna 'specs' en el DataFrame original con la nueva columna tratada
modelo_df['specs'] = specs_variable
# Imprimimos para ver como quedo la columna después de reemplazar
print(modelo_df['specs'])

0                                            Single-player
1        Single-player, Multi-player, Online Multi-Play...
2        Single-player, Multi-player, Online Multi-Play...
3                                            Single-player
4        Single-player, Full controller support, HTC Vi...
                               ...                        
32128                    Single-player, Steam Achievements
32129    Single-player, Steam Achievements, Steam Cloud...
32130    Single-player, Steam Achievements, Steam Tradi...
32131       Single-player, Steam Achievements, Steam Cloud
32132    Single-player, Stats, Steam Leaderboards, HTC ...
Name: specs, Length: 32133, dtype: object


In [4]:
# Vamos a cambiar unos errores que hay en la columna specs
# como vemos hay unos errores de caracteres, que vamos a solucionar de la siguiente manera
#modelo_df['specs'] = modelo_df['specs'].replace('Design &amp; Illustration', 'Design & Illustration')
#modelo_df['specs'] = modelo_df['specs'].replace('Animation &amp; Modeling', 'Animation & Modeling')

In [7]:
print(modelo_df['specs'].unique())

['Single-player'
 'Single-player, Multi-player, Online Multi-Player, Cross-Platform Multiplayer, Steam Achievements, Steam Trading Cards, In-App Purchases'
 'Single-player, Multi-player, Online Multi-Player, In-App Purchases, Stats'
 ...
 'Steam Workshop, Steam Cloud, HTC Vive, Oculus Rift, Windows Mixed Reality, Tracked Motion Controllers, Keyboard / Mouse, Seated, Standing, Room-Scale'
 'Single-player, Multi-player, Online Multi-Player, Online Co-op, Partial Controller Support, Stats'
 'Multi-player, Co-op, Shared/Split Screen, Steam Achievements, Full controller support, Steam Trading Cards']


Vamos a empezar con el modelo de machine learning

CountVectorizer es una clase de la biblioteca scikit-learn que se utiliza para convertir una colección de documentos de texto en una matriz de recuento de términos o tokens. 

Toma un conjunto de documentos de texto y crea una representación numérica de esos documentos en forma de una matriz donde cada fila representa un documento y cada columna representa una palabra única en el conjunto de documentos. La entrada en cada celda de la matriz indica cuántas veces aparece esa palabra en el documento correspondiente.

In [8]:
# Cargamos en la variable cv, CountVectorizer que es la herramienta de preprocesamiento que convierte datos 
# de texto en una representación numérica adecuada para ser utilizada por algoritmos de aprendizaje automático
cv = CountVectorizer()

In [9]:
# ahora vamos a ajustar la herramienta CountVectorizer al contenido de la columna 'specs' 
# y transformar esos datos en una matriz de recuento de términos
# Con .toarray(): Convierte la representación dispersa de la matriz de recuento de términos a una matriz densa.
# .shape: Devuelve la forma (número de filas y columnas) de la matriz resultante.
cv.fit_transform(modelo_df['specs']).toarray().shape

(32133, 71)

In [10]:
# Se generan los vectores a comparar 
vectores = cv.fit_transform(modelo_df['specs']).toarray()

In [11]:
# Mostramos como quedo Nuestra variable vectores
vectores

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=int64)

cosine_similarity() nos va a servir para Calcular la similitud coseno entre todos los pares de vectores en la matriz.

La similitud coseno es una medida de similitud entre dos vectores en un espacio multidimensional que mide el coseno del ángulo entre ellos. Se utiliza comúnmente para comparar la similitud entre documentos en el análisis de texto.

In [12]:
# Aplicamos la similitud del coseno a nuestros vectores
similitud = cosine_similarity(vectores)

In [13]:
# Se obtiene el array de similitud
similitud[0]

array([1.        , 0.53452248, 0.64888568, ..., 0.47140452, 0.5       ,
       0.36514837])

In [14]:
# Ordenamos la similitud entre más similar a menos similar tomando 5 valores
sorted(list(enumerate(similitud[0])), reverse=True, key=lambda x:x[1])[1:6]

[(3, 0.9999999999999998),
 (10, 0.9999999999999998),
 (13, 0.9999999999999998),
 (14, 0.9999999999999998),
 (21, 0.9999999999999998)]

Ahora que tenemos la matriz ya ordenada y tomamos los valores que necesitamos, vamos a generar una funcion para poder despues crear una columna y guardar los datos para agilizar la funcion consulta

In [17]:
modelo_df["item_id"] = modelo_df["item_id"].astype(int)


In [19]:
# Se genera una función que te da la recomendación por título
def recomendacion(juego):
    # Obtenemos el índice del juego con el ID dado
    indice_juego = modelo_df[modelo_df["item_id"] == juego].index[0]

    # Obtenemos las distancias de similitud para el juego dado
    distances = similitud[indice_juego]

    # Ordenamos las distancias en orden descendente y obtener las 5 recomendaciones principales (excluyendo el juego dado)
    lista_juegos = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]

    # Obtenemos los nombres de los juegos recomendados
    recommended_titles = [modelo_df.iloc[i[0]]['name'] for i in lista_juegos]

    return recommended_titles

Vamos a aplicar la funcion al dataframe modelo_df para crear la columna RecomendacionesTop5 con todas las recomendaciones que nos arrojo el modelo y la funcion

In [20]:
modelo_df

,specs,item_id,name
0,Single-player,761140,Lost Summoner Kitty
1,"Single-player, Multi-player, Online Multi-Play...",643980,Ironbound
2,"Single-player, Multi-player, Online Multi-Play...",670290,Real Pool 3D - Poolians
3,Single-player,767400,弹炸人2222
4,"Single-player, Full controller support, HTC Vi...",773570,Log Challenge
...,...,...,...
32128,"Single-player, Steam Achievements",773640,Colony On Mars
32129,"Single-player, Steam Achievements, Steam Cloud...",733530,LOGistICAL: South Africa
32130,"Single-player, Steam Achievements, Steam Tradi...",610660,Russian Roads
32131,"Single-player, Steam Achievements, Steam Cloud",658870,EXIT 2 - Directions


In [21]:
# Se aplica la función al dataframe para obtener una nueva columna con las recomendaciones ya que es más facil 
# de cargar y leer para la funcion
modelo_df['RecomendacionesTop5'] = modelo_df['item_id'].apply(recomendacion)

In [22]:
# Vemos como nos quedaron las columnas y el peso del dataframe
modelo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   specs                32133 non-null  object
 1   item_id              32133 non-null  int32 
 2   name                 32132 non-null  object
 3   RecomendacionesTop5  32133 non-null  object
dtypes: int32(1), object(3)
memory usage: 878.8+ KB


In [23]:
modelo_df

,specs,item_id,name,RecomendacionesTop5
0,Single-player,761140,Lost Summoner Kitty,"[弹炸人2222, Uncanny Islands, Beach Rules, Planet..."
1,"Single-player, Multi-player, Online Multi-Play...",643980,Ironbound,"[Duelyst, Warhammer 40,000: Regicide, KROSMAGA..."
2,"Single-player, Multi-player, Online Multi-Play...",670290,Real Pool 3D - Poolians,"[Heroes of Havoc: Idle Adventures, Tactical Mo..."
3,Single-player,767400,弹炸人2222,"[弹炸人2222, Uncanny Islands, Beach Rules, Planet..."
4,"Single-player, Full controller support, HTC Vi...",773570,Log Challenge,"[Jam Session VR, The Trace, Caretaker Retribut..."
...,...,...,...,...
32128,"Single-player, Steam Achievements",773640,Colony On Mars,[Army of Tentacles: (Not) A Cthulhu Dating Sim...
32129,"Single-player, Steam Achievements, Steam Cloud...",733530,LOGistICAL: South Africa,"[Runespell: Overture, Rush for Glory, BoomTown..."
32130,"Single-player, Steam Achievements, Steam Tradi...",610660,Russian Roads,"[Drawn®: The Painted Tower, Tropico 4, The Bin..."
32131,"Single-player, Steam Achievements, Steam Cloud",658870,EXIT 2 - Directions,"[Fate of the World, Fate of the World: Tipping..."


In [24]:
# Una vez echa la columna de recomendaciones podemos borrar los demas campos que no precisamos
modelo_df.drop(columns=['name','specs'], inplace=True)

In [25]:
# Chequeamos
modelo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   item_id              32133 non-null  int32 
 1   RecomendacionesTop5  32133 non-null  object
dtypes: int32(1), object(1)
memory usage: 376.7+ KB


Por ultimo creamos el CSV para alimentar la funcion consulta

In [26]:
# Guardamos el resultado en un nuevo archivo CSV
modelo_df.to_csv('modelo_espec.csv', index=False)

In [27]:
modelo_df

,item_id,RecomendacionesTop5
0,761140,"[弹炸人2222, Uncanny Islands, Beach Rules, Planet..."
1,643980,"[Duelyst, Warhammer 40,000: Regicide, KROSMAGA..."
2,670290,"[Heroes of Havoc: Idle Adventures, Tactical Mo..."
3,767400,"[弹炸人2222, Uncanny Islands, Beach Rules, Planet..."
4,773570,"[Jam Session VR, The Trace, Caretaker Retribut..."
...,...,...
32128,773640,[Army of Tentacles: (Not) A Cthulhu Dating Sim...
32129,733530,"[Runespell: Overture, Rush for Glory, BoomTown..."
32130,610660,"[Drawn®: The Painted Tower, Tropico 4, The Bin..."
32131,658870,"[Fate of the World, Fate of the World: Tipping..."
